In [1]:
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
import os
import pandas as pd
import warnings
from tqdm import tqdm  # Import tqdm for progress bars

In [2]:
# Suppress specific NodataWarning without importing rasterstats.io
warnings.filterwarnings("ignore", message="Setting nodata to -999; specify nodata explicitly")

# Set nodata explicitly in the zonal_stats call (e.g., nodata=-999 or another value)
nodata_value = -999  # You can specify a different nodata value if needed


In [3]:
# Load the shapefile with circular polygons
shapefile_path = 'F:/Landcover Classification_North Greece/Buffer/Buffer.shp'
polygons = gpd.read_file(shapefile_path)

In [4]:
# Calculate centroids and add them to the GeoDataFrame
polygons['centroid'] = polygons.geometry.centroid
polygons['centroid_x'] = polygons['centroid'].x
polygons['centroid_y'] = polygons['centroid'].y

In [5]:
# Base folder containing all subdirectories with raster TIFF files
base_raster_folder = 'F:/Landcover Classification_North Greece/Raster_files'

In [6]:
# Initialize a list to store the results
results = []

In [7]:
# Get total number of folders and raster files for progress tracking
raster_files = [os.path.join(root, file) for root, _, files in os.walk(base_raster_folder) for file in files if file.endswith('.tif')]
total_rasters = len(raster_files) * len(polygons)

In [8]:
# Initialize the tqdm progress bar
with tqdm(total=total_rasters, desc="Processing rasters and polygons") as pbar:

    # Loop over the subdirectories in the main raster folder
    for folder_name in os.listdir(base_raster_folder):
        folder_path = os.path.join(base_raster_folder, folder_name)
        
        if os.path.isdir(folder_path):  # Check if it's a directory
            # Loop over raster files in the subdirectory
            for raster_file in os.listdir(folder_path):
                if raster_file.endswith('.tif'):  # Only process TIFF files
                    raster_path = os.path.join(folder_path, raster_file)
                    
                    # Open the raster file
                    with rasterio.open(raster_path) as src:
                        
                        # Calculate statistics (min, max, mean) for each polygon, specifying the nodata value
                        stats = zonal_stats(polygons, raster_path, stats=['min', 'max', 'mean'], nodata=nodata_value)
                        
                        # Add each stat result along with the polygon attributes to the list
                        for i, stat in enumerate(stats):
                            # Extract the full row of attributes from the shapefile for each polygon
                            polygon_attributes = polygons.iloc[i].to_dict()  # Convert row to dictionary
                            
                            # Combine polygon attributes with the statistics for each polygon
                            result = {
                                **polygon_attributes,  # Include all the original shapefile attributes
                                'raster_file': raster_file,
                                'mean': stat['mean'],
                                'centroid_x': polygon_attributes['centroid_x'],  # Add centroid X coordinate
                                'centroid_y': polygon_attributes['centroid_y'],  # Add centroid Y coordinate
                                'VEG_CODE': polygon_attributes.get('VEG_CODE', None)  # Get VEG_CODE attribute if it exists
                            }
                            results.append(result)
                            
                            # Update the progress bar after processing each polygon
                            pbar.update(1)

Processing rasters and polygons: 100%|██████████| 1412208/1412208 [1:55:27<00:00, 203.86it/s]  


In [25]:
# Convert results into a DataFrame for easy saving or further processing
df = pd.DataFrame(results)

In [29]:
# Save to a CSV file
output_csv = 'F:/Landcover Classification_North Greece/output_statistics_multiple_folders.csv'
df.to_csv(output_csv, index=False)

print(f"Results saved to {output_csv}")

Results saved to F:/Landcover Classification_North Greece/output_statistics_multiple_folders.csv
